<a href="https://colab.research.google.com/github/Anavi07/AudioSpotting/blob/main/Russian_MS_All_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
import os
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import cv2
import warnings
warnings.filterwarnings("ignore")

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
!pip install kagglehub

In [47]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [48]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("railotrailot/speech-in-russian-digits-from-0-to-9")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/railotrailot/speech-in-russian-digits-from-0-to-9/versions/1


In [49]:
#!kaggle datasets download -d abdulkaderghandoura/arabic-speech-commands-dataset
!kaggle datasets download -d railotrailot/speech-in-russian-digits-from-0-to-9

#https://www.kaggle.com/datasets/railotrailot/speech-in-russian-digits-from-0-to-9/data

Dataset URL: https://www.kaggle.com/datasets/railotrailot/speech-in-russian-digits-from-0-to-9
License(s): copyright-authors
speech-in-russian-digits-from-0-to-9.zip: Skipping, found more recently modified local copy (use --force to force download)


In [50]:
import zipfile
import os

# Path to the uploaded zip file
zip_file_path = "/content/speech-in-russian-digits-from-0-to-9.zip"


# Directory where you want to extract the files
extract_dir = "/content/russian_speech_dataset"

# Create the directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Files extracted to: {extract_dir}")


Files extracted to: /content/russian_speech_dataset


In [51]:
# Paths
dataset_dir = '/content/russian_speech_dataset'  # Path to the dataset folder containing digit folders
output_dir = '/content/drive/MyDrive/russian'  # Path to save processed data

# Ensure output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [52]:
# PROCESSING
# Feature extraction methods

# def CalculateZeroCrossingRate(file_location):
#     y, sr = librosa.load(file_location)
#     zCross = librosa.feature.zero_crossing_rate(y=y)
#     dim = (32, 32)
#     resized = cv2.resize(zCross, dim, interpolation = cv2.INTER_AREA)
#     return resized

# def CalculateMelSpectrogram(file_location):
#     y, sr = librosa.load(file_location)
#     melSpec = librosa.feature.melspectrogram(y=y, sr=sr)
#     melSpec_dB = librosa.power_to_db(melSpec)
#     dim = (32, 32)
#     resized = cv2.resize(melSpec_dB, dim, interpolation = cv2.INTER_AREA)
#     return resized


def CalculateMFCC(file_location):
    y, sr = librosa.load(file_location)
    melSpec = librosa.feature.mfcc(y=y, sr=sr)
    dim = (32, 32)
    resized = cv2.resize(melSpec, dim, interpolation = cv2.INTER_AREA)
    return resized



In [53]:
cd /content

/content


In [54]:
'''for i in range(10):
  j = '/content/dataset'
  print(os.listdir(j))
  for k in os.listdir(j):
    print(k)
    if ".DS_" in k:
      continue

    for t in os.listdir(j+"/"+k):
      #print("path = ", os.listdir(j+"/"+k))
      if ".DS_" in k:
        continue
      print(t.split("WA")[1].split(".")[0])
      #print(t.split("D")[1][0])
      break
    break
  break '''

'for i in range(10):\n  j = \'/content/dataset\'\n  print(os.listdir(j))\n  for k in os.listdir(j):\n    print(k)\n    if ".DS_" in k:\n      continue\n\n    for t in os.listdir(j+"/"+k):\n      #print("path = ", os.listdir(j+"/"+k))\n      if ".DS_" in k:\n        continue\n      print(t.split("WA")[1].split(".")[0])\n      #print(t.split("D")[1][0])\n      break\n    break\n  break '

In [55]:
import os
import numpy as np

# Paths
#/content/russian_speech_dataset
main_dir = "/content/russian_speech_dataset/voice_0-9_RU"  # Path to the folder containing WAV files
files_path = "/content/processed_russian_speech"  # Path to save processed data

# Ensure output directory exists
if not os.path.exists(files_path):
    os.makedirs(files_path)

# Initialize data containers
x = []
y = []

# Loop through each file in the main directory
for file_name in os.listdir(main_dir):
    if not file_name.endswith(".wav"):  # Skip non-wav files
        continue

    # Extract label from the first character of the filename
    try:
        label = int(file_name[0])  # First character should be a digit
    except ValueError:
        print(f"Skipping file {file_name} as it does not start with a digit.")
        continue

    file_path = os.path.join(main_dir, file_name)

    try:
        # Extract Mel Spectrogram features
        x.append(CalculateMFCC(file_path))  # Replace with your feature extraction function
        y.append(label)  # Use the first character as the label
    except Exception as e:
        print(f"Error processing {file_name}: {e}")

# Convert data to NumPy arrays
x = np.array(x)
y = np.array(y)

# Verify processed data
if x.size == 0 or y.size == 0:
    raise ValueError("No data was processed. Check dataset structure and feature extraction.")

# Save processed data
np.save(os.path.join(files_path, "X_mfcc.npy"), x)
np.save(os.path.join(files_path, "Y_mfcc.npy"), y)

print(f"Processing completed. Processed data saved in {files_path}.")


Error processing 0-21-male-5776.wav: 
Error processing 6-23-male-9131.wav: 
Error processing 6-23-male-9134.wav: 
Error processing 3-21-male-4671.wav: 
Error processing 2-23-male-9522.wav: 
Error processing 4-23-male-3113.wav: 
Error processing 4-21-male-7232.wav: 
Error processing 5-23-male-3182.wav: 
Error processing 2-21-male-5946.wav: 
Error processing 5-21-male-5291.wav: 
Error processing 1-21-male-5074.wav: 
Error processing 7-21-male-6381.wav: 
Error processing 9-21-male-4970.wav: 
Error processing 0-21-male-5753.wav: 
Error processing 5-21-male-5289.wav: 
Error processing 7-21-male-6417.wav: 
Error processing 5-21-male-4796.wav: 
Error processing 3-21-male-3998.wav: 
Error processing 3-21-male-3994.wav: 
Error processing 0-23-male-2776.wav: 
Error processing 1-23-male-8888.wav: 
Error processing 1-21-male-6662.wav: 
Error processing 7-23-male-3377.wav: 
Error processing 6-21-male-5634.wav: 
Error processing 4-21-male-6788.wav: 
Error processing 3-21-male-4023.wav: 
Processing c

In [56]:
# import os
# import numpy as np

# # Paths
# main_dir = "/content/arabic_speech_dataset/dataset/dataset"  # Path to the main dataset folder
# files_path = "/content/drive/MyDrive/arabic"  # Path to save processed data

# # Ensure output directory exists
# if not os.path.exists(files_path):
#     os.makedirs(files_path)

# # Helper function to convert folder names to labels
# def folder_to_label(folder_name):
#     arabic_to_english = {
#         "zero": 0,
#         "one": 1,
#         "two": 2,
#         "three": 3,
#         "four": 4,
#         "five": 5,
#         "six": 6,
#         "seven": 7,
#         "eight": 8,
#         "nine": 9,
#     }
#     return arabic_to_english.get(folder_name.lower())

# # Initialize data containers
# x = []
# y = []

# # Loop through each folder in the main directory
# for label_dir in os.listdir(main_dir):
#     label_path = os.path.join(main_dir, label_dir)

#     # Skip non-digit folders and non-directories
#     label = folder_to_label(label_dir)
#     if label is None or not os.path.isdir(label_path):
#         continue

#     # Process audio files within the label directory
#     for file_name in os.listdir(label_path):
#         if not file_name.endswith(".wav"):  # Skip non-wav files
#             continue

#         file_path = os.path.join(label_path, file_name)

#         try:
#             # Extract Mel Spectrogram features
#             x.append(CalculateMelSpectrogram(file_path))  # Replace with your feature extraction function
#             y.append(label)  # Use folder name as the label
#         except Exception as e:
#             print(f"Error processing {file_name}: {e}")

# # Convert data to NumPy arrays
# x = np.array(x)
# y = np.array(y)

# # Verify processed data
# if x.size == 0 or y.size == 0:
#     raise ValueError("No data was processed. Check dataset structure and feature extraction.")

# # Save processed data
# np.save(os.path.join(files_path, "X_melspectrogram.npy"), x)
# np.save(os.path.join(files_path, "Y_melspectrogram.npy"), y)

# print(f"Processing completed. Processed data saved in {files_path}.")


In [57]:
#load files
#/content/processed_russian_speech
files_path = "/content/processed_russian_speech"  # Save processed data

#x_melspectrogram = np.load(files_path+"/X_melspectrogram.npy", allow_pickle=True)
# x_zero_crossing_rate = np.load(files_path+"/X_zero_crossing_rate.npy", allow_pickle=True)
x_mfcc = np.load(files_path+"/X_mfcc.npy", allow_pickle=True)
#y_melspectrogram = np.load(files_path+"/Y_melspectrogram.npy", allow_pickle=True)
# y_zero_crossing_rate = np.load(files_path+"/y_zero_crossing_rate.npy", allow_pickle=True)
y_mfcc = np.load(files_path+"/Y_mfcc.npy", allow_pickle=True)

In [58]:
#Split data into train and test

from sklearn.model_selection import train_test_split
x = x_mfcc
y = y_mfcc

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)
np.save(files_path+"/x_mfcc_train",x_train)
np.save(files_path+"/y_mfcc_train",y_train)
np.save(files_path+"/x_mfcc_test",x_test)
np.save(files_path+"/y_mfcc_test",y_test)

In [59]:
# from sklearn.model_selection import train_test_split
# x = x_melspectrogram
# y = y_melspectrogram

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)
# np.save(files_path+"/x_melspectrogram_train",x_train)
# np.save(files_path+"/y_melspectrogram_train",y_train)
# np.save(files_path+"/x_melspectrogram_test",x_test)
# np.save(files_path+"/y_melspectrogram_test",y_test)

In [60]:

# from sklearn.model_selection import train_test_split
# x = x_zero_crossing_rate
# y = y_zero_crossing_rate

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)
# np.save(files_path+"/x_zero_crossing_rate_train",x_train)
# np.save(files_path+"/y_zero_crossing_rate_train",y_train)
# np.save(files_path+"/x_zero_crossing_rate_test",x_test)
# np.save(files_path+"/y_zero_crossing_rate_test",y_test)

In [61]:
import tensorflow as tf
import numpy as np

files_path = "/content/processed_russian_speech"

features = ['melspectrogram', 'mfcc', 'zero_crossing_rate']

x_train = np.load(files_path+"/x_{}_train.npy".format(features[1]), allow_pickle=True)
y_train = np.load(files_path+"/y_{}_train.npy".format(features[1]), allow_pickle=True)

x_test = np.load(files_path+"/x_{}_test.npy".format(features[1]),allow_pickle=True)
y_test = np.load(files_path+"/y_{}_test.npy".format(features[1]),allow_pickle=True)

In [62]:
print(np.unique(y_mfcc))

[0 1 2 3 4 5 6 7 8 9]


In [63]:
x_train =x_train.reshape((x_train.shape[0],32,32,1))
x_test =x_test.reshape((x_test.shape[0],32,32,1))

## CNN

In [64]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
def GetCNN():
  model = models.Sequential()

  model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(32, 32, 1)))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.BatchNormalization())

  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.BatchNormalization())

  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.BatchNormalization())

  model.add(layers.Dropout(0.1))
  model.add(layers.Flatten())
  model.add(layers.Dense(256, activation='relu'))# fully connected
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(128, activation='relu'))
  model.add(layers.Dropout(0.1))

  model.add(layers.Dense(10,activation='softmax', use_bias=True))
  return model

In [65]:
from sklearn.preprocessing import LabelBinarizer
label_as_binary = LabelBinarizer()
train_y = label_as_binary.fit_transform(y_train)
test_y = label_as_binary.fit_transform(y_test)

In [66]:
# Normalise
x_train = x_train / 255.0
x_test = x_test / 255.0

In [67]:
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf

# Ensure consistency by fitting the LabelBinarizer only once on all possible labels
label_as_binary = LabelBinarizer()
label_as_binary.fit(np.unique(y_melspectrogram))  # Fit on all unique labels (0–9)

# Transform train and test labels
train_y = label_as_binary.transform(y_train)
test_y = label_as_binary.transform(y_test)

# Verify shapes
print("X train shape = ", x_train.shape)
print("y train shape = ", train_y.shape)
print("X test shape = ", x_test.shape)
print("y test shape = ", test_y.shape)

# CNN Model
cnn_model = GetCNN()
print(cnn_model.summary())

cnn_model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=["accuracy"],
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01, clipnorm=1.0),
)

# Define save path for the best model
save_path = '/content/drive/MyDrive/saved files/russian/models'
callbacks = ModelCheckpoint(
    save_path + "/melspectrogram_model_.keras",
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

# Train the model
history = cnn_model.fit(
    x_train, train_y,
    batch_size=64,
    epochs=50,
    verbose=1,
    validation_split=0.1,
    callbacks=[callbacks]
)

# Evaluate the model
print("Test Accuracy is ", cnn_model.evaluate(x_test, test_y)[1])


X train shape =  (7979, 32, 32, 1)
y train shape =  (7979, 10)
X test shape =  (1995, 32, 32, 1)
y test shape =  (1995, 10)


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_29 (Conv2D)                   │ (None, 30, 30, 64)          │             640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_11 (MaxPooling2D)      │ (None, 15, 15, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_9                │ (None, 15, 15, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_30 (Conv2D)                   │ (None, 13, 13, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_12 (MaxPooling2D)      │ (None, 6, 6, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_10               │ (None, 6, 6, 64)            │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_31 (Conv2D)                   │ (None, 4, 4, 64)            │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_13 (MaxPooling2D)      │ (None, 2, 2, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_11               │ (None, 2, 2, 64)            │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 2, 2, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 256)                 │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 175,242 (684.54 KB)

 Trainable params: 174,858 (683.04 KB)

 Non-trainable params: 384 (1.50 KB)

None
Epoch 1/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.7044 - loss: 0.9058
Epoch 1: val_accuracy improved from -inf to 0.31328, saving model to /content/drive/MyDrive/saved files/russian/models/melspectrogram_model_.keras
113/113 ━━━━━━━━━━━━━━━━━━━━ 16s 54ms/step - accuracy: 0.7058 - loss: 0.9017 - val_accuracy: 0.3133 - val_loss: 2.4075
Epoch 2/50
104/113 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9575 - loss: 0.1491
Epoch 2: val_accuracy improved from 0.31328 to 0.47870, saving model to /content/drive/MyDrive/saved files/russian/models/melspectrogram_model_.keras
113/113 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.9580 - loss: 0.1477 - val_accuracy: 0.4787 - val_loss: 2.0512
Epoch 3/50
101/113 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9685 - loss: 0.1131
Epoch 3: val_accuracy improved from 0.47870 to 0.85714, saving model to /content/drive/MyDrive/saved files/russian/models/melspectrogram_model_.keras
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 

In [68]:
# Load the saved model
cnn_model = tf.keras.models.load_model(save_path + "/melspectrogram_model_.keras")

# Evaluate on the training set
train_metrics = cnn_model.evaluate(x_train, train_y, verbose=1)
print(f"Train Loss: {train_metrics[0]}, Train Accuracy: {train_metrics[1]}")

# Evaluate on the test set
test_metrics = cnn_model.evaluate(x_test, test_y, verbose=1)
print(f"Test Loss: {test_metrics[0]}, Test Accuracy: {test_metrics[1]}")

# If validation data was split during training (validation_split=0.1)
x_val = x_train[-int(0.1 * len(x_train)):]
y_val = train_y[-int(0.1 * len(train_y)):]

val_metrics = cnn_model.evaluate(x_val, y_val, verbose=1)
print(f"Validation Loss: {val_metrics[0]}, Validation Accuracy: {val_metrics[1]}")

250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9974 - loss: 0.0112
Train Loss: 0.012081852182745934, Train Accuracy: 0.9971174597740173
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9969 - loss: 0.0179
Test Loss: 0.014721316285431385, Test Accuracy: 0.9974937438964844
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9978 - loss: 0.0135
Validation Loss: 0.0141533724963665, Validation Accuracy: 0.9974905848503113


In [69]:
# import matplotlib.pyplot as plt

# plt.plot(history.history['loss'], label='Training Loss')
# plt.plot(history.history['val_loss'], label='Validation Loss')
# plt.legend()
# plt.show()


In [70]:
# import keras
# zcr_model = keras.models.load_model(save_path+"/{}_model_.keras".format(features[0]))
# print("Test Accuracy is ", cnn_model.evaluate(x_test,test_y)[1])

## GoogleNet

In [71]:
import tensorflow as tf
from tensorflow.keras import layers, models

def GetGoogleNet():
    input_layer = layers.Input(shape=(32, 32, 1))

    # First Convolutional Block
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(input_layer)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.BatchNormalization()(x)

    # Inception Module 1
    branch1 = layers.Conv2D(32, (1, 1), activation='relu', padding='same')(x)

    branch2 = layers.Conv2D(32, (1, 1), activation='relu', padding='same')(x)
    branch2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(branch2)

    branch3 = layers.Conv2D(32, (1, 1), activation='relu', padding='same')(x)
    branch3 = layers.Conv2D(64, (5, 5), activation='relu', padding='same')(branch3)

    branch4 = layers.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch4 = layers.Conv2D(32, (1, 1), activation='relu', padding='same')(branch4)

    x = layers.Concatenate()([branch1, branch2, branch3, branch4])
    x = layers.BatchNormalization()(x)

    # Inception Module 2
    branch1 = layers.Conv2D(64, (1, 1), activation='relu', padding='same')(x)

    branch2 = layers.Conv2D(64, (1, 1), activation='relu', padding='same')(x)
    branch2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(branch2)

    branch3 = layers.Conv2D(64, (1, 1), activation='relu', padding='same')(x)
    branch3 = layers.Conv2D(128, (5, 5), activation='relu', padding='same')(branch3)

    branch4 = layers.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch4 = layers.Conv2D(64, (1, 1), activation='relu', padding='same')(branch4)

    x = layers.Concatenate()([branch1, branch2, branch3, branch4])
    x = layers.BatchNormalization()(x)

    # Global Average Pooling + Fully Connected Layers
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    output_layer = layers.Dense(10, activation='softmax')(x)

    model = models.Model(inputs=input_layer, outputs=output_layer)
    return model


In [72]:
# Model Compilation and Training
save_path = '/content/drive/MyDrive/saved files/russian/models'

cnn_model = GetGoogleNet()
print(cnn_model.summary())

cnn_model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=["accuracy"],
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01, clipnorm=1.0),
)

from tensorflow.keras.callbacks import ModelCheckpoint
callbacks = ModelCheckpoint(
    save_path + "/googleNet_ms_model.keras",
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

print("X train shape = ", x_train.shape)
print("y train shape = ", train_y.shape)

history = cnn_model.fit(
    x_train, train_y,
    batch_size=64,
    epochs=50,
    verbose=1,
    validation_split=0.1,
    callbacks=[callbacks]
)

Model: "functional_116"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_10            │ (None, 32, 32, 1)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_32 (Conv2D)        │ (None, 32, 32, 64)     │            640 │ input_layer_10[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_14          │ (None, 16, 16, 64)     │              0 │ conv2d_32[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_12    │ (None, 16, 16, 64)     │            256 │ max_pooling2d_14[0][0] │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_34 (Conv2D)        │ (None, 16, 16, 32)     │          2,080 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_36 (Conv2D)        │ (None, 16, 16, 32)     │          2,080 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_15          │ (None, 16, 16, 64)     │              0 │ batch_normalization_1… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_33 (Conv2D)        │ (None, 16, 16, 32)     │          2,080 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_35 (Conv2D)        │ (None, 16, 16, 64)     │         18,496 │ conv2d_34[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_37 (Conv2D)        │ (None, 16, 16, 64)     │         51,264 │ conv2d_36[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_38 (Conv2D)        │ (None, 16, 16, 32)     │          2,080 │ max_pooling2d_15[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_2             │ (None, 16, 16, 192)    │              0 │ conv2d_33[0][0],       │
│ (Concatenate)             │                        │                │ conv2d_35[0][0],       │
│                           │                        │                │ conv2d_37[0][0],       │
│                           │                        │                │ conv2d_38[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_13    │ (None, 16, 16, 192)    │            768 │ concatenate_2[0][0]    │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_40 (Conv2D)        │ (None, 16, 16, 64)     │         12,352 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_42 (Conv2D)        │ (None, 16, 16, 64)     │         12,352 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_16     

 Total params: 542,218 (2.07 MB)

 Trainable params: 540,938 (2.06 MB)

 Non-trainable params: 1,280 (5.00 KB)

None
X train shape =  (7979, 32, 32, 1)
y train shape =  (7979, 10)
Epoch 1/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.5739 - loss: 1.1369
Epoch 1: val_accuracy improved from -inf to 0.09900, saving model to /content/drive/MyDrive/saved files/russian/models/googleNet_ms_model.keras
113/113 ━━━━━━━━━━━━━━━━━━━━ 27s 105ms/step - accuracy: 0.5755 - loss: 1.1328 - val_accuracy: 0.0990 - val_loss: 168.5888
Epoch 2/50
110/113 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9156 - loss: 0.2676
Epoch 2: val_accuracy did not improve from 0.09900
113/113 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.9164 - loss: 0.2652 - val_accuracy: 0.0990 - val_loss: 185.6587
Epoch 3/50
109/113 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9637 - loss: 0.1119
Epoch 3: val_accuracy did not improve from 0.09900
113/113 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.9638 - loss: 0.1116 - val_accuracy: 0.0990 - val_loss: 87.7438
Epoch 4/50
109/113 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy:

In [73]:
# Reshape the train and test data back to (32, 32, 1)
x_train_reshaped = tf.image.resize(x_train, (32, 32))  # Resize to match original input
x_test_reshaped = tf.image.resize(x_test, (32, 32))    # Resize to match original input

# Add grayscale channel
x_train_reshaped = tf.expand_dims(x_train_reshaped, axis=-1)
x_test_reshaped = tf.expand_dims(x_test_reshaped, axis=-1)

In [74]:
# Load the saved model
googleNet_model = tf.keras.models.load_model(save_path + "/googleNet_ms_model.keras")

# Evaluate on the training set
train_metrics = googleNet_model.evaluate(x_train_reshaped, train_y, verbose=1)
print(f"Train Loss: {train_metrics[0]}, Train Accuracy: {train_metrics[1]}")

# Evaluate on the test set
test_metrics = googleNet_model.evaluate(x_test_reshaped, test_y, verbose=1)
print(f"Test Loss: {test_metrics[0]}, Test Accuracy: {test_metrics[1]}")

# If validation data was split during training (validation_split=0.1)
x_val_reshaped = x_train_reshaped[-int(0.1 * len(x_train_reshaped)):]
y_val = train_y[-int(0.1 * len(train_y)):]

val_metrics = googleNet_model.evaluate(x_val_reshaped, y_val, verbose=1)
print(f"Validation Loss: {val_metrics[0]}, Validation Accuracy: {val_metrics[1]}")

250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9948 - loss: 0.0255
Train Loss: 0.035782936960458755, Train Accuracy: 0.9947361946105957
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9902 - loss: 0.0308
Test Loss: 0.03545756638050079, Test Accuracy: 0.9929824471473694
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.9931 - loss: 0.0986
Validation Loss: 0.07650607824325562, Validation Accuracy: 0.9962359070777893


## LeNet

In [75]:
def GetLeNet():
    model = models.Sequential()

    # Layer 1
    model.add(layers.Conv2D(6, (5, 5), activation='relu', input_shape=(32, 32, 1), padding='same'))
    model.add(layers.AveragePooling2D((2, 2)))

    # Layer 2
    model.add(layers.Conv2D(16, (5, 5), activation='relu'))
    model.add(layers.AveragePooling2D((2, 2)))

    # Flatten and Dense layers
    model.add(layers.Flatten())
    model.add(layers.Dense(120, activation='relu'))
    model.add(layers.Dense(84, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))  # 10 classes

    return model


In [76]:
save_path = '/content/drive/MyDrive/saved files/russian/models'

# Instantiate the model
lenet_model = GetLeNet()

# Compile the model
lenet_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

# Print the model summary
print(lenet_model.summary())

# Train the model
from tensorflow.keras.callbacks import ModelCheckpoint
callbacks = ModelCheckpoint(save_path + "/lenet_ms_model.keras", monitor='val_accuracy',
                            mode='max', save_best_only=True, verbose=1)

# Train the AlexNet model
history = lenet_model.fit(
    x_train, train_y,
    validation_split=0.1,
    batch_size=64,
    epochs=50,
    callbacks=[callbacks],
    verbose=1
)

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_45 (Conv2D)                   │ (None, 32, 32, 6)           │             156 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d_2                  │ (None, 16, 16, 6)           │               0 │
│ (AveragePooling2D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_46 (Conv2D)                   │ (None, 12, 12, 16)          │           2,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ average_pooling2d_3                  │ (None, 6, 6, 16)            │               0 │
│ (AveragePooling2D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ (None, 576)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_22 (Dense)                     │ (None, 120)                 │          69,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_23 (Dense)                     │ (None, 84)                  │          10,164 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_24 (Dense)                     │ (None, 10)                  │             850 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 82,826 (323.54 KB)

 Trainable params: 82,826 (323.54 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.4259 - loss: 1.7847
Epoch 1: val_accuracy improved from -inf to 0.89599, saving model to /content/drive/MyDrive/saved files/russian/models/lenet_ms_model.keras
113/113 ━━━━━━━━━━━━━━━━━━━━ 7s 42ms/step - accuracy: 0.4278 - loss: 1.7792 - val_accuracy: 0.8960 - val_loss: 0.3389
Epoch 2/50
110/113 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9243 - loss: 0.2642
Epoch 2: val_accuracy improved from 0.89599 to 0.94862, saving model to /content/drive/MyDrive/saved files/russian/models/lenet_ms_model.keras
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9249 - loss: 0.2627 - val_accuracy: 0.9486 - val_loss: 0.1529
Epoch 3/50
111/113 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9664 - loss: 0.1306
Epoch 3: val_accuracy improved from 0.94862 to 0.97494, saving model to /content/drive/MyDrive/saved files/russian/models/lenet_ms_model.keras
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9665 - loss: 0.1302 

In [77]:
# Load the saved model
leNet_model = tf.keras.models.load_model(save_path + "/lenet_ms_model.keras")

# Evaluate on the training set
train_metrics = leNet_model.evaluate(x_train_reshaped, train_y, verbose=1)
print(f"Train Loss: {train_metrics[0]}, Train Accuracy: {train_metrics[1]}")

# Evaluate on the test set
test_metrics = leNet_model.evaluate(x_test_reshaped, test_y, verbose=1)
print(f"Test Loss: {test_metrics[0]}, Test Accuracy: {test_metrics[1]}")

# If validation data was split during training (validation_split=0.1)
x_val_reshaped = x_train_reshaped[-int(0.1 * len(x_train_reshaped)):]
y_val = train_y[-int(0.1 * len(train_y)):]

val_metrics = leNet_model.evaluate(x_val_reshaped, y_val, verbose=1)
print(f"Validation Loss: {val_metrics[0]}, Validation Accuracy: {val_metrics[1]}")

250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9998 - loss: 0.0019
Train Loss: 0.0019294910598546267, Train Accuracy: 0.9998746514320374
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9960 - loss: 0.0095
Test Loss: 0.0077722626738250256, Test Accuracy: 0.9969924688339233
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 1.0000 - loss: 0.0059
Validation Loss: 0.003854548092931509, Validation Accuracy: 1.0


## MobileNetV2

In [78]:
# import tensorflow as tf
# from tensorflow.keras import layers, models

# # Preprocess the data to match MobileNet input requirements
# import numpy as np
# x_train_3ch = np.repeat(x_train, 3, axis=-1)  # Convert (32, 32, 1) to (32, 32, 3)
# x_test_3ch = np.repeat(x_test, 3, axis=-1)

# # Define the MobileNet model
# def GetMobileNet():
#     base_model = tf.keras.applications.MobileNetV2(
#         input_shape=(32, 32, 3),
#         include_top=False,
#         weights=None  # Training from scratch for audio classification
#     )

#     model = tf.keras.Sequential([
#         base_model,
#         tf.keras.layers.GlobalAveragePooling2D(),
#         tf.keras.layers.Dense(128, activation='relu'),
#         tf.keras.layers.Dropout(0.5),
#         tf.keras.layers.Dense(10, activation='softmax')  # Adjust for 10-class classification
#     ])

#     return model


In [79]:
# save_path = '/content/drive/MyDrive/saved files/russian/models'

# # Instantiate the model
# mobileNet_model = GetMobileNet()

# # Compile the model
# mobileNet_model.compile(
#     optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
#     loss=tf.keras.losses.CategoricalCrossentropy(),
#     metrics=['accuracy']
# )

# # Print the model summary
# print(mobileNet_model.summary())

# # Train the model
# from tensorflow.keras.callbacks import ModelCheckpoint
# callbacks = ModelCheckpoint(
#     save_path + "/mobileNet_ms_model.keras",
#     monitor='val_accuracy',
#     mode='max',
#     save_best_only=True,
#     verbose=1
# )

# # Train the MobileNet model
# history = mobileNet_model.fit(
#     x_train_3ch, train_y,
#     validation_split=0.1,
#     batch_size=64,
#     epochs=50,
#     callbacks=[callbacks],
#     verbose=1
# )


In [80]:
# # Load the saved model
# mobileNet_model = tf.keras.models.load_model(save_path + "/mobileNet_ms_model.keras")

# # Evaluate on the training set
# train_metrics = mobileNet_model.evaluate(x_train_3ch, train_y, verbose=1)
# print(f"Train Loss: {train_metrics[0]}, Train Accuracy: {train_metrics[1]}")

# # Evaluate on the test set
# test_metrics = mobileNet_model.evaluate(x_test_3ch, test_y, verbose=1)
# print(f"Test Loss: {test_metrics[0]}, Test Accuracy: {test_metrics[1]}")

# # If validation data was split during training (validation_split=0.1)
# x_val_reshaped = x_train_3ch[-int(0.1 * len(x_train_3ch)):]
# y_val = train_y[-int(0.1 * len(train_y)):]

# val_metrics = mobileNet_model.evaluate(x_val_reshaped, y_val, verbose=1)
# print(f"Validation Loss: {val_metrics[0]}, Validation Accuracy: {val_metrics[1]}")


## EfficientNetB0

In [81]:
def GetEfficientNet():
    base_model = tf.keras.applications.EfficientNetB0(
        input_shape=(32, 32, 3),
        include_top=False,
        weights=None
    )

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ])

    return model

In [82]:
save_path = '/content/drive/MyDrive/saved files/russian/models'

efficientnet_model = GetEfficientNet()

# Compile the model
efficientnet_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

# Print the model summary
print(efficientnet_model.summary())

# Train the model
from tensorflow.keras.callbacks import ModelCheckpoint
callbacks = ModelCheckpoint(save_path + "/efficientnet_ms_model.keras", monitor='val_accuracy',
                            mode='max', save_best_only=True, verbose=1)

# Train the AlexNet model
history = efficientnet_model.fit(
    x_train, train_y,
    validation_split=0.1,
    batch_size=64,
    epochs=50,
    callbacks=[callbacks],
    verbose=1
)

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)          │ (None, 1, 1, 1280)          │       4,049,571 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_5           │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_25 (Dense)                     │ (None, 128)                 │         163,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_14 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_26 (Dense)                     │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,214,829 (16.08 MB)

 Trainable params: 4,172,806 (15.92 MB)

 Non-trainable params: 42,023 (164.16 KB)

None
Epoch 1/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 279ms/step - accuracy: 0.4136 - loss: 1.7563
Epoch 1: val_accuracy improved from -inf to 0.09900, saving model to /content/drive/MyDrive/saved files/russian/models/efficientnet_ms_model.keras
113/113 ━━━━━━━━━━━━━━━━━━━━ 115s 372ms/step - accuracy: 0.4155 - loss: 1.7508 - val_accuracy: 0.0990 - val_loss: 2.3564
Epoch 2/50
112/113 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.8802 - loss: 0.3810
Epoch 2: val_accuracy did not improve from 0.09900
113/113 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.8805 - loss: 0.3802 - val_accuracy: 0.0990 - val_loss: 2.5130
Epoch 3/50
112/113 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9373 - loss: 0.2341
Epoch 3: val_accuracy improved from 0.09900 to 0.12406, saving model to /content/drive/MyDrive/saved files/russian/models/efficientnet_ms_model.keras
113/113 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - accuracy: 0.9374 - loss: 0.2337 - val_accuracy: 0.1241 - val_loss: 2.6510
Epoch 4/50
112/113 ━━━━━━━━━

In [83]:
# Load the saved model
efficientnet_model = tf.keras.models.load_model(save_path + "/efficientnet_ms_model.keras")

# Evaluate on the training set
train_metrics = efficientnet_model.evaluate(x_train, train_y, verbose=1)
print(f"Train Loss: {train_metrics[0]}, Train Accuracy: {train_metrics[1]}")

# Evaluate on the test set
test_metrics = efficientnet_model.evaluate(x_test, test_y, verbose=1)
print(f"Test Loss: {test_metrics[0]}, Test Accuracy: {test_metrics[1]}")

# If validation data was split during training (validation_split=0.1)
x_val = x_train[-int(0.1 * len(x_train)):]
y_val = train_y[-int(0.1 * len(train_y)):]

val_metrics = efficientnet_model.evaluate(x_val, y_val, verbose=1)
print(f"Validation Loss: {val_metrics[0]}, Validation Accuracy: {val_metrics[1]}")

250/250 ━━━━━━━━━━━━━━━━━━━━ 13s 22ms/step - accuracy: 0.9993 - loss: 0.0042
Train Loss: 0.0029151751659810543, Train Accuracy: 0.9994986653327942
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9978 - loss: 0.0078
Test Loss: 0.007018954027444124, Test Accuracy: 0.9984962344169617
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 151ms/step - accuracy: 0.9985 - loss: 0.0048
Validation Loss: 0.004969767294824123, Validation Accuracy: 0.998745322227478


## AlexNet

In [84]:
import tensorflow as tf
from tensorflow.keras import layers, models

def GetAlexNet():
    model = models.Sequential([
        # First Convolutional Layer
        layers.Conv2D(96, (3, 3), strides=1, activation='relu', input_shape=(32, 32, 3), padding='same'),
        layers.MaxPooling2D(pool_size=(2, 2), strides=2),
        layers.BatchNormalization(),

        # Second Convolutional Layer
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D(pool_size=(2, 2), strides=2),
        layers.BatchNormalization(),

        # Third, Fourth, Fifth Convolutional Layers
        layers.Conv2D(384, (3, 3), activation='relu', padding='same'),
        layers.Conv2D(384, (3, 3), activation='relu', padding='same'),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D(pool_size=(2, 2), strides=2),
        layers.BatchNormalization(),

        # Fully Connected Layers
        layers.Flatten(),
        layers.Dense(4096, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(4096, activation='relu'),
        layers.Dropout(0.5),

        # Output Layer
        layers.Dense(10, activation='softmax')  # Adjust for 10-class classification
    ])

    return model

In [85]:
save_path = '/content/drive/MyDrive/saved files/russian/models'

# Preprocess the data to match AlexNet input requirements
x_train_3ch = np.repeat(x_train, 3, axis=-1)  # Convert (32, 32, 1) to (32, 32, 3)
x_test_3ch = np.repeat(x_test, 3, axis=-1)

# Instantiate the AlexNet model
alexnet_model = GetAlexNet()

# Compile the model
alexnet_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

# Print the model summary
print(alexnet_model.summary())

# Train the model
from tensorflow.keras.callbacks import ModelCheckpoint
callbacks = ModelCheckpoint(
    save_path + "/alexNet_ms_model.keras",
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

# Train the AlexNet model
history = alexnet_model.fit(
    x_train_3ch, train_y,
    validation_split=0.1,
    batch_size=64,
    epochs=50,
    callbacks=[callbacks],
    verbose=1
)

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_47 (Conv2D)                   │ (None, 32, 32, 96)          │           2,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_17 (MaxPooling2D)      │ (None, 16, 16, 96)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_15               │ (None, 16, 16, 96)          │             384 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_48 (Conv2D)                   │ (None, 16, 16, 256)         │         221,440 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_18 (MaxPooling2D)      │ (None, 8, 8, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_16               │ (None, 8, 8, 256)           │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_49 (Conv2D)                   │ (None, 8, 8, 384)           │         885,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_50 (Conv2D)                   │ (None, 8, 8, 384)           │       1,327,488 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_51 (Conv2D)                   │ (None, 8, 8, 256)           │         884,992 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_19 (MaxPooling2D)      │ (None, 4, 4, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_17               │ (None, 4, 4, 256)           │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_5 (Flatten)                  │ (None, 4096)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_27 (Dense)                     │ (None, 4096)                │      16,781,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_15 (Dropout)                 │ (None, 4096)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_28 (Dense)                     │ (None, 4096)                │      16,781,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_16 (Dropout)                 │ (None, 4096)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_29 (Dense)                     │ (None, 10)                  │          40,970 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 36,927,754 (140.87 MB)

 Trainable params: 36,926,538 (140.86 MB)

 Non-trainable params: 1,216 (4.75 KB)

None
Epoch 1/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.3768 - loss: 3.8410
Epoch 1: val_accuracy improved from -inf to 0.10150, saving model to /content/drive/MyDrive/saved files/russian/models/alexNet_ms_model.keras
113/113 ━━━━━━━━━━━━━━━━━━━━ 27s 125ms/step - accuracy: 0.3788 - loss: 3.8220 - val_accuracy: 0.1015 - val_loss: 31.1105
Epoch 2/50
112/113 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.8987 - loss: 0.3407
Epoch 2: val_accuracy did not improve from 0.10150
113/113 ━━━━━━━━━━━━━━━━━━━━ 8s 39ms/step - accuracy: 0.8991 - loss: 0.3395 - val_accuracy: 0.1015 - val_loss: 25.8351
Epoch 3/50
111/113 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9564 - loss: 0.1405
Epoch 3: val_accuracy did not improve from 0.10150
113/113 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - accuracy: 0.9565 - loss: 0.1406 - val_accuracy: 0.1015 - val_loss: 45.0183
Epoch 4/50
111/113 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.9658 - loss: 0.1435
Epoch 4: val_accuracy improved from 0.10150 

In [86]:
# Load the saved model
alexNet_model = tf.keras.models.load_model(save_path + "/alexNet_ms_model.keras")

# Evaluate on the training set
train_metrics = alexNet_model.evaluate(x_train_3ch, train_y, verbose=1)
print(f"Train Loss: {train_metrics[0]}, Train Accuracy: {train_metrics[1]}")

# Evaluate on the test set
test_metrics = alexNet_model.evaluate(x_test_3ch, test_y, verbose=1)
print(f"Test Loss: {test_metrics[0]}, Test Accuracy: {test_metrics[1]}")

# If validation data was split during training (validation_split=0.1)
x_val_reshaped = x_train_3ch[-int(0.1 * len(x_train_3ch)):]
y_val = train_y[-int(0.1 * len(train_y)):]

val_metrics = alexNet_model.evaluate(x_val_reshaped, y_val, verbose=1)
print(f"Validation Loss: {val_metrics[0]}, Validation Accuracy: {val_metrics[1]}")


250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9967 - loss: 0.0160
Train Loss: 0.013366921804845333, Train Accuracy: 0.9971174597740173
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9972 - loss: 0.0090
Test Loss: 0.011413563042879105, Test Accuracy: 0.9969924688339233
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9993 - loss: 0.0095
Validation Loss: 0.01610821671783924, Validation Accuracy: 0.998745322227478


## SqueezeNet

In [87]:
import tensorflow as tf
from tensorflow.keras import layers, models

def GetSqueezeNet():
    model = models.Sequential([
        layers.Conv2D(96, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)),
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),

        layers.Conv2D(10, (1, 1), activation='softmax'),  # Adjust for 10-class classification
        layers.GlobalAveragePooling2D()  # Global pooling to reduce dimensions
    ])

    return model

In [88]:
save_path = '/content/drive/MyDrive/saved files/russian/models'

# Preprocess the data to match SqueezeNet input requirements
x_train_3ch = np.repeat(x_train, 3, axis=-1)  # Convert (32, 32, 1) to (32, 32, 3)
x_test_3ch = np.repeat(x_test, 3, axis=-1)

# Instantiate the SqueezeNet model
squeezenet_model = GetSqueezeNet()

# Compile the model
squeezenet_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

# Print the model summary
print(squeezenet_model.summary())

# Train the model
from tensorflow.keras.callbacks import ModelCheckpoint
callbacks = ModelCheckpoint(
    save_path + "/squeezeNet_model.keras",
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

# Train the SqueezeNet model
history = squeezenet_model.fit(
    x_train_3ch, train_y,
    validation_split=0.1,
    batch_size=64,
    epochs=50,
    callbacks=[callbacks],
    verbose=1
)

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_52 (Conv2D)                   │ (None, 32, 32, 96)          │           2,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_20 (MaxPooling2D)      │ (None, 16, 16, 96)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_53 (Conv2D)                   │ (None, 16, 16, 128)         │         110,720 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_54 (Conv2D)                   │ (None, 16, 16, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_21 (MaxPooling2D)      │ (None, 8, 8, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_55 (Conv2D)                   │ (None, 8, 8, 256)           │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_56 (Conv2D)                   │ (None, 8, 8, 256)           │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_57 (Conv2D)                   │ (None, 8, 8, 10)            │           2,570 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_6           │ (None, 10)                  │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,148,810 (4.38 MB)

 Trainable params: 1,148,810 (4.38 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.2471 - loss: 2.0212
Epoch 1: val_accuracy improved from -inf to 0.56642, saving model to /content/drive/MyDrive/saved files/russian/models/squeezeNet_model.keras
113/113 ━━━━━━━━━━━━━━━━━━━━ 8s 38ms/step - accuracy: 0.2485 - loss: 2.0181 - val_accuracy: 0.5664 - val_loss: 1.2472
Epoch 2/50
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6837 - loss: 1.0650
Epoch 2: val_accuracy improved from 0.56642 to 0.87719, saving model to /content/drive/MyDrive/saved files/russian/models/squeezeNet_model.keras
113/113 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.6844 - loss: 1.0638 - val_accuracy: 0.8772 - val_loss: 0.7307
Epoch 3/50
111/113 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8882 - loss: 0.6594
Epoch 3: val_accuracy improved from 0.87719 to 0.94110, saving model to /content/drive/MyDrive/saved files/russian/models/squeezeNet_model.keras
113/113 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.8889 - los

In [89]:
# Load the saved model
squeezeNet_model = tf.keras.models.load_model(save_path + "/squeezeNet_model.keras")

# Evaluate on the training set
train_metrics = squeezeNet_model.evaluate(x_train_3ch, train_y, verbose=1)
print(f"Train Loss: {train_metrics[0]}, Train Accuracy: {train_metrics[1]}")

# Evaluate on the test set
test_metrics = squeezeNet_model.evaluate(x_test_3ch, test_y, verbose=1)
print(f"Test Loss: {test_metrics[0]}, Test Accuracy: {test_metrics[1]}")

# If validation data was split during training (validation_split=0.1)
x_val_reshaped = x_train_3ch[-int(0.1 * len(x_train_3ch)):]
y_val = train_y[-int(0.1 * len(train_y)):]

val_metrics = squeezeNet_model.evaluate(x_val_reshaped, y_val, verbose=1)
print(f"Validation Loss: {val_metrics[0]}, Validation Accuracy: {val_metrics[1]}")


250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9994 - loss: 0.0841
Train Loss: 0.08415597677230835, Train Accuracy: 0.9996240139007568
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9997 - loss: 0.0870
Test Loss: 0.08608689904212952, Test Accuracy: 0.999498724937439
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 1.0000 - loss: 0.0975
Validation Loss: 0.09431435167789459, Validation Accuracy: 1.0


## ResNet50

In [90]:
# import tensorflow as tf
# from tensorflow.keras import models, layers
# from tensorflow.keras.applications import ResNet50  # Import ResNet50
# from tensorflow.keras.callbacks import ModelCheckpoint

# # Resize and convert the data from grayscale to RGB
# x_train_resized = tf.image.resize(x_train, (224, 224))  # Resize to 224x224
# x_train_rgb = tf.image.grayscale_to_rgb(x_train_resized)  # Convert to 3 channels

# x_test_resized = tf.image.resize(x_test, (224, 224))
# x_test_rgb = tf.image.grayscale_to_rgb(x_test_resized)  # Convert to 3 channels

# # Verify the shapes of the transformed data
# print(f"x_train_rgb shape: {x_train_rgb.shape}")
# print(f"x_test_rgb shape: {x_test_rgb.shape}")


In [91]:
# # Function to create a ResNet50 model for audio digit classification
# def GetResNet50():
#     # Load the ResNet50 architecture without the top classification layer
#     base_model = ResNet50(
#         include_top=False,
#         weights=None,  # No pretrained weights since this is specific to audio digits
#         input_shape=(224, 224, 3)  # Adjusted input shape for ResNet50
#     )

#     # Add custom classification layers on top of ResNet50
#     model = models.Sequential()
#     model.add(base_model)
#     model.add(layers.GlobalAveragePooling2D())
#     model.add(layers.Dense(256, activation='relu'))  # Fully connected layer
#     model.add(layers.Dropout(0.2))
#     model.add(layers.Dense(128, activation='relu'))
#     model.add(layers.Dropout(0.2))
#     model.add(layers.Dense(10, activation='softmax'))  # 10 classes for digit classification

#     return model

In [92]:
# save_path = '/content/drive/MyDrive/saved files/arabic/models'

# # Create the ResNet50 model
# resnet_model = GetResNet50()
# print(resnet_model.summary())

# # Compile the model
# resnet_model.compile(
#     loss=tf.keras.losses.CategoricalCrossentropy(),
#     metrics=["accuracy"],
#     optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, clipnorm=1.0)
# )

# # Define the model checkpoint callback
# callbacks = ModelCheckpoint(
#     save_path + "/resnet50_ms_model.keras",
#     monitor='val_accuracy',
#     mode='max',
#     save_best_only=True,
#     verbose=1
# )

# # Print shapes of input and output
# print("X train shape = ", x_train_rgb.shape)
# print("y train shape = ", train_y.shape)

# # Train the model
# history = resnet_model.fit(
#     x_train_rgb, train_y,
#     batch_size=64,
#     epochs=50,
#     verbose=1,
#     validation_split=0.1,
#     callbacks=[callbacks]
# )

In [93]:
# # Load the saved model
# resnet_model = tf.keras.models.load_model(save_path + "/resnet50_ms_model.keras")

# # Evaluate on the training set
# train_metrics = resnet_model.evaluate(x_train_rgb, train_y, verbose=1)
# print(f"Train Loss: {train_metrics[0]}, Train Accuracy: {train_metrics[1]}")

# # Evaluate on the test set
# test_metrics = resnet_model.evaluate(x_test_rgb, test_y, verbose=1)
# print(f"Test Loss: {test_metrics[0]}, Test Accuracy: {test_metrics[1]}")

# # If validation data was split during training (validation_split=0.1)
# x_val = x_train[-int(0.1 * len(x_train)):]
# y_val = train_y[-int(0.1 * len(train_y)):]

# val_metrics = resnet_model.evaluate(x_val, y_val, verbose=1)
# print(f"Validation Loss: {val_metrics[0]}, Validation Accuracy: {val_metrics[1]}")

In [94]:
# # Load all models
# import keras

# mp_model = keras.models.load_model(save_path+"/melspectrogram_model_.keras")
# mfcc_model = keras.models.load_model(save_path+"/mfcc_model_.keras")
# zc_model = keras.models.load_model(save_path+"/zero_crossing_rate_model_.keras")

In [95]:
# # Model Ensembling
# from keras.models import Model
# from keras import layers

# def ensembleModels(models, model_input):
#     # collect outputs of models in a list
#     model_outputs=[model(model_input) for model in models]
#     # averaging outputs
#     ensemble_output=layers.average(model_outputs)
#     # build model from same input and avg output
#     modelEns = Model(inputs=model_input, outputs=ensemble_output,    name='ensemble')

#     return modelEns

In [96]:
# from keras.layers import Input  # Correct import for Input
# from keras.models import Model  # Ensure you have this for model creation

# # Assuming ensembleModels is defined elsewhere
# models = [mp_model, mfcc_model, zc_model]

# model_input = Input(shape=models[0].input_shape[1:])  # c*h*w
# modelEns = ensembleModels(models, model_input)  # Assuming this function is implemented
# modelEns.summary()

# modelEns.save(save_path + "/ensemble_.keras", include_optimizer=True)

In [97]:
# from keras.models import load_model

# # Load the model
# model = load_model(save_path + "/ensemble_.keras")

# # Compile the model (use the same loss and metrics as during training)
# model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# # Print the model summary
# model.summary()

# # Evaluate the model
# test_accuracy = model.evaluate(x_test, test_y)[1]
# print("Test Accuracy is ", test_accuracy)


In [98]:
# print("Metrics:", model.metrics_names)  # This should list the metrics


NameError: name 'model' is not defined

In [ ]:
# from sklearn.preprocessing import LabelBinarizer

# def evaluateAll(model, x_test, test_y):
#   label_as_binary = LabelBinarizer()
#   test_y = label_as_binary.fit_transform(test_y)
#   return model.evaluate(x_test,test_y)



In [ ]:
# #Test data
# import keras

# path = '/content/drive/MyDrive/processed_data/audd'

# x_test = np.load(path+"/x_melspectrogram_test.npy",allow_pickle=True)
# y_test = np.load(path+"/y_melspectrogram_test.npy",allow_pickle=True)


# mp_model = keras.models.load_model(save_path+"/melspectrogram_model_.keras")

# model_result = evaluateAll(mp_model, x_test, y_test)
# print(model_result)
# print("model history ", mp_model.history)